In [2]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
os.environ['USE_PYGEOS'] = '0'
import json
import math
import matplotlib.pyplot as plt
import fiona
import pyproj
import contextily
import plotly.express as px
import dash_mantine_components as dmc


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
path = os.getcwd()
#path = "/srv/data/my_shared_data_folder/cpr/"

#dir_list = os.listdir(path)
#dir_in = os.listdir(os.getcwd())
#directory = pd.Series(dir_in)
#print(dir_in)
#print(dir_list)

In [3]:
features = pd.read_csv('../data/pesticide_5_w_tracts+.csv', low_memory = False, index_col = 0) 

In [18]:
# this is loading in the file from a google drive
# this is applicable when the file is too large, get the share link, go to it, attempt download
# then inspect the download button, grab the link from there and this will work

features = pd.read_csv('https://drive.google.com/u/0/uc?id=1TnotAGr6PRQx3uq6l9tGtRbDvFfC0PRY&export=download&confirm=t&uuid=6bdfadff-8ff3-4b94-ab26-49d5fdb4b3bb&at=ALt4Tm0afFCSipw4dElGheLpAbrK:1689730220309', index_col=0)

In [3]:
features

,YEAR,COUNTY_NAME,COMTRS,POUNDS_PRODUCT_APPLIED,AMOUNT_TREATED,AERIAL_GROUND_INDICATOR,Meridian,County_Code,Section,air,carcinogens,endo_disrupt,ch_I,fumigants,neonicotinoids,repro_dev,TownshipRange,COUNTYFP,TRACTCE,TRACTFP,PU_FIPS,MTownshipRange,Count
0,2018,FRESNO,10M11S13E15,486.7475,69.8,G,MDM,10,15,True,False,False,False,False,False,False,T11S R13E,19,8402,8402,6019008402,MDMT11S R13E,1
1,2018,FRESNO,10M11S13E15,69.1503,69.8,G,MDM,10,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T11S R13E,19,8402,8402,6019008402,MDMT11S R13E,1
2,2018,FRESNO,10M11S13E15,126.9997,69.8,G,MDM,10,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T11S R13E,19,8402,8402,6019008402,MDMT11S R13E,1
3,2018,FRESNO,10M11S13E15,0.2139,69.8,G,MDM,10,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T11S R13E,19,8402,8402,6019008402,MDMT11S R13E,1
4,2018,FRESNO,10M11S13E15,22.0036,69.8,G,MDM,10,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T11S R13E,19,8402,8402,6019008402,MDMT11S R13E,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4842576,2021,MONTEREY,27M20S09E15,80.0000,10.0,G,MDM,27,15,True,False,False,False,True,False,False,T20S R09E,53,11305,11305,6053011305,MDMT20S R09E,1
4842577,2021,MONTEREY,27M21S10E28,200.0000,599.0,G,MDM,27,28,True,False,False,False,True,False,False,T21S R10E,53,11400,11400,6053011400,MDMT21S R10E,1
4842578,2021,MONTEREY,27M15S01E21,0.0003,1.0,O,MDM,27,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T15S R01E,53,13300,13300,6053013300,MDMT15S R01E,1
4842579,2021,MONTEREY,27M15S01E21,0.0001,1.0,O,MDM,27,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T15S R01E,53,13300,13300,6053013300,MDMT15S R01E,1


In [3]:
# residual, currently not needed
def Cfp_to_CfpString(num):
    Cfp = str(num)
    if (len(Cfp) == 1):
        fix = '00' + Cfp
    elif (len(Cfp) == 2):
        fix = '0' + Cfp
    elif (len(Cfp) == 3):
        fix = Cfp
    else: return np.nan

    return fix

def tract_fixer(num):
    num = str(num)
    if (len(num) >= 2) & (len(num) <=6):
        zero_times = 6 - len(num)
        fix = zero_times*'0' + num
        return fix
    else: return np.nan

In [4]:
# open to plot tract level
place1 = gpd.read_file('../data/tracts_w_geo+.geojson')
f = open('../data/tracts_w_geo+.geojson')
places1 = json.load(f) 

In [3]:
place1.head(2)

,STATEFP,COUNTYFP,TRACTFP,FIPS GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,FIPS,state,tract,geoid,area_name,area_land,total_pop,housing_units,owner_occupied_housing_units,persons_no_health_insurance,families,families_income_below_poverty_level,median_household_income,nh_white_alone,nh_black_or_african_american_alone,nh_american_indian_and_alaska_native_alone,nh_asian_alone,nh_native_hawaiian_and_other_pacific_islander_alone,nh_other,nh_two_or_more,hispanic_or_latino,households_receiving_assistance,households,pct_owner_occupied,pct_no_health_insurance,pct_families_income_below_poverty_level,pct_households_receiving_assistance,pct_nh_white_alone,pct_nh_black_or_african_american_alone,pct_nh_american_indian_and_alaska_native_alone,pct_nh_asian_alone,pct_nh_native_hawaiian_and_other_pacific_islander_alone,pct_nh_other,pct_nh_two_or_more,pct_hispanic_or_latino,geometry
0,06,085,504321,6085504321,5043.21,Census Tract 5043.21,G5020,S,1450237,0,+37.3931319,-121.8651427,06085504321,ca,504321,14000US06085504321,Census Tract 5043.21,0.559940,5511,1779,1131,212,1316,60,146941.0,544,77,0,3559,0,7,269,1055,962,1675,0.635750,0.038469,0.045593,0.574328,0.098712,0.013972,0.000000,0.645799,0.0,0.00127,0.048811,0.191435,"POLYGON ((-121.87556 37.39924, -121.87535 37.3..."
1,06,085,504410,6085504410,5044.10,Census Tract 5044.10,G5020,S,1102136,0,+37.4093719,-121.8788884,06085504410,ca,504410,14000US06085504410,Census Tract 5044.10,0.425537,4286,1244,688,189,934,102,128523.0,556,165,77,2476,0,0,94,918,589,1214,0.553055,0.044097,0.109208,0.485173,0.129725,0.038497,0.017965,0.577695,0.0,0.00000,0.021932,0.214186,"POLYGON ((-121.88886 37.40758, -121.88576 37.4..."


In [5]:
# plotting TownshipRange 
place2 = gpd.read_file('../data/places_w_MTownship.geojson')
g = open('../data/places_w_MTownship.geojson')
places2 = json.load(g)

In [10]:
place2.head(2)

,OBJECTID,Township,Range,Meridian,Source,TownshipRange,Shape_Leng,MTownshipRange,geometry
0,1,T01N,R12W,SBM,BLM,T01N R12W,0.061541,SBMT01N R12W,"POLYGON ((-118.17028 34.20826, -118.17433 34.2..."
1,2,T01N,R13W,SBM,DPR,T01N R13W,0.066493,SBMT01N R13W,"POLYGON ((-118.18727 34.16590, -118.18727 34.1..."


**Experimental Code**

In [6]:
# needed for creating dash options
di = dict(zip(list(features.COUNTY_NAME.unique()), list(features.COUNTY_NAME.unique())))
di2 = dict(zip(['','carcinogens','endo_disrupt','ch_I','fumigants','neonicotinoids','repro_dev'],['','carcinogens','endo_disrupt','ch_I','fumigants','neonicotinoids','repro_dev']))

In [1]:
# print(di)

In [2]:
# print(di2)

# Dash app

In [8]:
# Running dash app...
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)

app.layout = html.Div([
    html.Div([
    html.P("Select pesticide type:"),
    dcc.Dropdown(options = [{'label': v, 'value': k} for k, v in di2.items()],
                value =  '',
                id = 'pesticide_select',
                multi=False),
    html.P("Select plotting unit:"),
    dcc.RadioItems(
            id='geometry_select', 
            options=[{'label':'Township-Range','value':'TownshipRange'},{'label':'Tract','value':'tracts'}],
            value="TownshipRange",
            inline=True
        ),         
    ],
    style=dict(width='15%')),
    
    html.Div([
        html.H4('Pesticide Use Plotter'),
        html.P("Select statistic:"),
        dcc.RadioItems(
            id='column', 
            options=[{'label':'Total Pounds of Pesticide Used',"value":"POUNDS_PRODUCT_APPLIED"}, {"label":"Number of Incidences of Pesticide Use","value": "Count"}],
            value="POUNDS_PRODUCT_APPLIED",
            inline=True
        ),
        dcc.Graph(id="graph"),
        dcc.Dropdown(options = [{'label': v, 'value': k} for k, v in di.items()],
                       value =  ['FRESNO','KERN','MENDOCINO','MONTEREY','LOS ANGELES'],
                         id = 'county_multi_select',
                         multi=True),
        dcc.Slider(
                            2018,
                            2021,
                            step=1,
                            id='yearslider',
                            value=2020,
                            marks={str(year): str(year) for year in [2018,2019,2020,2021]},
        ),
        #dcc.Slider(
        #    2018,
        #    2021,
        #    step=1,
        #    id='yearslider',
        #    value=2020,
        #    marks={str(year): str(year) for year in features['YEAR'].unique()},
        #)
    ],
    style=dict(width='85%'))
],
style=dict(display='flex')
)
@app.callback(
    Output("graph", "figure"),
    Input("column", "value"),
    Input('county_multi_select', 'value'),
    Input('yearslider', 'value'),
    Input("pesticide_select", "value"),
    Input("geometry_select", "value")
    )#,
    #Input('County', 'value'))

def update_graph(column, county_multi_select, yearslider, pesticide_select, geometry_select):#County, yearslider):
    return plotr2(features, column, county = county_multi_select, year = yearslider, pesticide_type = pesticide_select,plot = geometry_select)#,county = [County], year = yearslider)

def plotr2(features, column, plot = 'TownshipRange', year = 2018,pesticide_type = '',county = ['FRESNO']): #change back later county = []):
    # 1. filtering for county
    if (county != []):
        features = features.loc[features['COUNTY_NAME'].isin(county)]
    
    # 2. filtering for year
    features = features.loc[features['YEAR'] == year]
    
    # 3. filtering for pesticide type
    if (pesticide_type != ''):
        if (pesticide_type in (list(features.columns))):
            features = features.loc[features[f'{pesticide_type}'] == True]
        else: print('invalid pesticide type')
    # 4. merging dataframe
    if (plot == 'tracts'):
        features = features.groupby(by = 'PU_FIPS').sum()[[column]]
        fig = px.choropleth_mapbox(features, 
                           geojson=places1,
                           featureidkey='properties.FIPS GEOID',
                           locations=features.index.to_series(), 
                           color=column,
                           color_continuous_scale="Viridis",
                           # range_color=(0,1_900_000),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat":36.746841 , "lon": -119.772591},
                           opacity=0.4,
                           #labels={'chem_type':'chemical type'}
                          )
   
    elif (plot == 'TownshipRange'):
        features = features.groupby(by = 'MTownshipRange').sum()[[column]]
        fig = px.choropleth_mapbox(features, 
                           geojson=places2,
                           featureidkey='properties.MTownshipRange',
                           locations=features.index.to_series(), 
                           color=column,
                           color_continuous_scale="Viridis",
                           #range_color=(0,2_000),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat":36.746841 , "lon": -119.772591},
                           opacity=0.4,
                           #labels={'chem_type':'chemical type'}
                          )
    else: return 'unable to plot'
        
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
        
    return fig

if __name__ == "__main__":
    app.run_server(debug=True)

# Just Function For Prototyping

In [ ]:
# function is designed for plotting any subset of counties in the dataframe, provided a county name column

# the column is sum aggregated by groupby and the this is the value that is displayed

# you can plot either by township range or tract, it will use different file to link the the correct file that maps to 
# the appropriate geometry

# year is selectable based on a year column

# pesticide type is selectable based on a pesticide type column

def plotr2(features, column, plot = 'TownshipRange', year = 2018,pesticide_type = '',county = ['FRESNO']): #change back later county = []):
    # 1. filtering for county
    if (county != []):
        features = features.loc[features['COUNTY_NAME'].isin(county)]
    
    # 2. filtering for year
    features = features.loc[features['YEAR'] == year]
    
    # 3. filtering for pesticide type
    if (pesticide_type != ''):
        if (pesticide_type in (list(features.columns))):
            features = features.loc[features[f'{pesticide_type}'] == True]
        else: print('invalid pesticide type')
    # 4. merging dataframe
    if (plot == 'tracts'):
        features = features.groupby(by = 'PU_FIPS').sum()[[column]]
        fig = px.choropleth_mapbox(features, 
                           geojson=places1,
                           featureidkey='properties.FIPS GEOID',
                           locations=features.index.to_series(), 
                           color=column,
                           color_continuous_scale="Viridis",
                           # range_color=(0,1_900_000),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat":36.746841 , "lon": -119.772591},
                           opacity=0.4,
                           #labels={'chem_type':'chemical type'}
                          )
   
    elif (plot == 'TownshipRange'):
        features = features.groupby(by = 'MTownshipRange').sum()[[column]]
        fig = px.choropleth_mapbox(features, 
                           geojson=places2,
                           featureidkey='properties.MTownshipRange',
                           locations=features.index.to_series(), 
                           color=column,
                           color_continuous_scale="Viridis",
                           #range_color=(0,2_000),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat":36.746841 , "lon": -119.772591},
                           opacity=0.4,
                           #labels={'chem_type':'chemical type'}
                          )
    else: return 'unable to plot'
        
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
        
    return fig